# Weighted Least Squares

In [ ]:
from gnss_lib_py.algorithms.snapshot import solve_wls
from gnss_lib_py.parsers.android import AndroidDerived2021
from gnss_lib_py.utils.visualizations import plot_metric

In [ ]:
# load Android Google Challenge data
!wget https://raw.githubusercontent.com/Stanford-NavLab/gnss_lib_py/main/data/unit_test/android_2021/Pixel4XL_derived.csv --quiet -O "Pixel4XL_derived.csv"
derived_data = AndroidDerived2021("Pixel4XL_derived.csv", remove_timing_outliers=False)

Solve for the Weighted Least Squares position estimate simply by passing the measurement data.

In [ ]:
state = solve_wls(derived_data)

Plot the ECEF x and ECEF y computed position estimate of the receiver

In [ ]:
fig = plot_metric(state, "x_rx_m", "y_rx_m", save=False)

# Residuals

In [ ]:
from gnss_lib_py.algorithms.residuals import solve_residuals
from gnss_lib_py.utils.visualizations import plot_metric_by_constellation

Solve for residuals using the estimated state. A new "residuals" row is added to derived_data

In [ ]:
solve_residuals(derived_data, state, inplace=True)

Plot the residuals using the `plot_residuals()` function. The residuals are broken up constellation and signal type and plotted on separate figures.

In [ ]:
figs = plot_metric_by_constellation(derived_data, "gps_millis", "residuals", save=False)